# First Steps --- Importing, and determining input and output path

In [562]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [563]:
import os 
import pandas as pd 
import numpy as np
import string
from datetime import datetime
input_path = 'gdrive/MyDrive/2020_21_2/Temalabor_1/raw_popdata/'
output_path = 'gdrive/MyDrive/2020_21_2/Temalabor_1/processed_popdata/'
files = os.listdir(input_path)

# Function

In [564]:
# Function 1.: Generating filesNames_dict dictionary where keys are in 'df_lakossag_yyyy' ("yyyy" = years) 
# and those keys refer to the actual filename that we want to read with pandas later on 

def key_dict_setup(keys, files_excel, fileNames_dict):
  for f in files_excel:
    if 'kozerdeku' in f:
      if 'xlsx' in f:
        keys.append('df'+ f[9:-5])
      else: 
        keys.append('df'+ f[9:-4])
    else:
      keys.append('df'+ '_' + f[:-4])      

  for i in range(len(keys)): 
    fileNames_dict[keys[i]] = files_excel[i]

  return keys, fileNames_dict

In [565]:
# Function 2.: Modifying columns which involve strings, namely: capitalize, applying strings to all columns including numerical columns, 
##trimming all leading and trailing whitesapces from str columns, 

def stringCol_mod(df_dict, *key):

  capitalize = lambda x: x.capitalize() if isinstance(x, str) else x
  trim_strings = lambda x: x.strip() if isinstance(x, str) else x

  try:
    df_dict[key[0]] = df_dict[key[0]].applymap(capitalize) 
    df_dict[key[0]] = df_dict[key[0]].applymap(str) #converting the whole df datatype into string
    df_dict[key[0]] = df_dict[key[0]].applymap(trim_strings)

    return df_dict[key[0]]

  except IndexError: 
    #df_dict = df_dict.applymap(capitalize)
    df_dict = df_dict.applymap(str) #converting the whole df datatype into string
    df_dict = df_dict.applymap(trim_strings)

    return df_dict



In [566]:
# Function 3.: From our original excel based dataset we only keep columns referring to the settlement and the pertaining population 
## Trimming all unnecessary starting columns at the end

def trim_numOfCol(df_dict, key): 

  # examine the # of columns in the dataframe
  if len(df_dict[key].columns) == 8:                # if # of cols ==  8
    df_dict[key] = df_dict[key].iloc[:, [3,7]]      # only preserve the 3rd and 7th column which are the name of the settlement and the population number, respectively.

  elif len(df_dict[key].columns) == 7:              # if # of cols ==  7
    df_dict[key] = df_dict[key].iloc[:, [2,6]]      # only preserve the 2nd and 6th column which are the name of the settlement and the population number, respectively.

  elif len(df_dict[key].columns) == 6:              # if # of cols ==  6
    df_dict[key] = df_dict[key].iloc[:, [1,5]]      # only preserve the 1st and 5th column which are the name of the settlement and the population number, respectively.

  
  colname_mapping = {df_dict[key].columns[0]:'name', df_dict[key].columns[1]: 'value'}   # only 2 columns remain, and rename them by using 'name' for the settlement and 'value' for the population # of that settlement, respecitvely. 
  df_dict[key] = df_dict[key].rename(columns = colname_mapping)


  if df_dict[key].iloc[1:2, :].name.str.contains('Budapest').bool(): 
    df_dict[key].drop(df_dict[key].index[[0]], inplace=True)
    df_dict[key].reset_index(drop=True, inplace=True)

  else: 
    df_dict[key].drop(df_dict[key].index[[0,1]], inplace=True)
    df_dict[key].reset_index(drop=True, inplace=True)

  return df_dict[key]



In [567]:
# Function 4.: Adding year and datatype==1650 to the appropriate population dataframe based on the year the excel file refers to. 

def add_datatypeid_year(df_dict, key):
  df_dict[key]['datatypeid'] = '1650' #Adding the right datatypeid
  df_dict[key]['year'] = key[-4:] #Adding the 2018 year
  cols = df_dict[key].columns.tolist() # listing all the columns, year is the last column at this moment 
  cols = cols[-1:] + cols[:-1]  #putting year as a first col 
  df_dict[key] = df_dict[key][cols] #save this into the pop2018 df 

  return df_dict[key]


In [568]:
# Function 5.: Pushing the IXth district row after the row pertaining to the IVth district (because of the order of bpDistrictGeoname dataset about Budapest after ordering it)

def BP_row_shifting(df_dict, key):

  idx = [i for i in range(len(df_dict[key]))]
  del idx[8]
  idx.insert(4,8)
  del idx[18]
  idx.insert(14,18)

  df_dict[key] = df_dict[key].iloc[idx].reset_index(drop=True)


  return df_dict[key]


In [569]:
# Function 6.: # Using bpDistrictGeoname to tag bpDistrict_pop then combine it with the appropriate dataframe

def BP_district_matching(df_dict, key, bpDistrictGeoname):
  bpDistrict_pop = df_dict[key][df_dict[key].name.str.contains('Budapest')]

  if len(bpDistrict_pop) == 22: 
    bpDistrict_pop.name = bpDistrictGeoname[0:22]

  elif len(bpDistrict_pop) == 23:
    bpDistrict_pop.name = bpDistrictGeoname

  df_dict[key] = bpDistrict_pop.combine_first(df_dict[key])

  return df_dict[key]


# Running the script 

In [570]:
#Listing all the excel files 

files_excel = [f for f in files if f[-3:] == 'xls' or f[-4:] == 'xlsx']
print(files_excel)

['kozerdeku_lakossag_2018.xlsx', 'kozerdeku_lakossag_2014.xls', 'lakossag_2010.xls', 'lakossag_2006.xls', 'lakossag_2002.xls', 'lakossag_1998.xls', 'lakossag_1994.xls', 'lakossag_1990.xls', 'kozerdeku_lakossag_2020.xlsx', 'kozerdeku_lakossag_2019.xlsx', 'kozerdeku_lakossag_2017.xlsx', 'kozerdeku_lakossag_2016.xls', 'kozerdeku_lakossag_2015.xls', 'kozerdeku_lakossag_2013.xls', 'kozerdeku_lakossag_2012.xls', 'kozerdeku_lakossag_2011.xls', 'lakossag_2009.xls', 'lakossag_2008.xls', 'lakossag_2007.xls', 'lakossag_2005.xls', 'lakossag_2004.xls', 'lakossag_2003.xls', 'lakossag_2001.xls', 'lakossag_2000.xls', 'lakossag_1999.xls', 'lakossag_1997.xls', 'lakossag_1996.xls', 'lakossag_1995.xls', 'lakossag_1993.xls', 'lakossag_1992.xls', 'lakossag_1991.xls', 'kozerdeku_lakossag_2021.xlsx', 'lakossag_1989.xls', 'lakossag_1988.xls', 'lakossag_1987.xls', 'lakossag_1986.xls']


In [571]:
###Using Function 2.

keys = list()
fileNames_dict = {}
keys, fileNames_dict = key_dict_setup(keys, files_excel, fileNames_dict)
keys = sorted(keys)

In [572]:
print(keys)

['df_lakossag_1986', 'df_lakossag_1987', 'df_lakossag_1988', 'df_lakossag_1989', 'df_lakossag_1990', 'df_lakossag_1991', 'df_lakossag_1992', 'df_lakossag_1993', 'df_lakossag_1994', 'df_lakossag_1995', 'df_lakossag_1996', 'df_lakossag_1997', 'df_lakossag_1998', 'df_lakossag_1999', 'df_lakossag_2000', 'df_lakossag_2001', 'df_lakossag_2002', 'df_lakossag_2003', 'df_lakossag_2004', 'df_lakossag_2005', 'df_lakossag_2006', 'df_lakossag_2007', 'df_lakossag_2008', 'df_lakossag_2009', 'df_lakossag_2010', 'df_lakossag_2011', 'df_lakossag_2012', 'df_lakossag_2013', 'df_lakossag_2014', 'df_lakossag_2015', 'df_lakossag_2016', 'df_lakossag_2017', 'df_lakossag_2018', 'df_lakossag_2019', 'df_lakossag_2020', 'df_lakossag_2021']


In [573]:
datatype = pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/datatype.csv', dtype=str)
datatype = stringCol_mod(datatype) #Using Function 2 
datatype[datatype['kshkod'] == 'TAAA200'] # For determining the size of a settlement, we need this: datatypeid==1650, kshkod==TAAA200

,datatypeid,kshkod,label,parentid
1270,1650,TAAA200,állandó népesség száma,2887


In [ ]:
geoname = pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/geoname.csv', dtype=str)
geoname = stringCol_mod(geoname) #Using Function 2 
bpDistrictGeoname = geoname[geoname['name'].str.match('Budapest')]['name'].sort_values()[1:] #the 0th one is just simply Budapest, and we just mine and sort all the name which contain Budapest districts
bpDistrictGeoname.reset_index(drop=True, inplace=True)

In [577]:
geoname.head(3)

,geonameid,name,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,6,Békésszentandrás,69,46.8709316036724,20.4842314849147
1,7,Békéssámson,2367,46.4224802344297,20.6200450334599
2,8,Bátaszék,672,46.1945947217959,18.7221321891037


In [ ]:
bpDistrictGeoname

0         Budapest I. kerület
1        Budapest II. kerület
2       Budapest III. kerület
3        Budapest IV. kerület
4        Budapest IX. kerület
5         Budapest V. kerület
6        Budapest VI. kerület
7       Budapest VII. kerület
8      Budapest VIII. kerület
9         Budapest X. kerület
10       Budapest XI. kerület
11      Budapest XII. kerület
12     Budapest XIII. kerület
13      Budapest XIV. kerület
14      Budapest XIX. kerület
15       Budapest XV. kerület
16      Budapest XVI. kerület
17     Budapest XVII. kerület
18    Budapest XVIII. kerület
19       Budapest XX. kerület
20      Budapest XXI. kerület
21     Budapest XXII. kerület
22    Budapest XXIII. kerület
Name: name, dtype: object

In [ ]:
df_dict = {}

population_data_basic = None
population_data_full = None

for key in keys:

  df_dict[key] = pd.read_excel(input_path + fileNames_dict[key], dtype=str)

  df_dict[key] = stringCol_mod(df_dict, key) #Using Function 2
  df_dict[key] = trim_numOfCol(df_dict, key) #Using Function 3
  df_dict[key] = add_datatypeid_year(df_dict, key)  #Using Function 4
  df_dict[key] = BP_row_shifting(df_dict, key) #Using Function5
  df_dict[key] = BP_district_matching(df_dict, key, bpDistrictGeoname) #Using Function6

  #Without any merge concat df_dict[key]
  population_data_basic = pd.concat([population_data_basic, df_dict[key]]) 
   
  #Merge than concat data 
  merged_df_dict_key = pd.merge(df_dict[key], geoname, on='name')
  population_data_full = pd.concat([population_data_full, merged_df_dict_key])

population_data_basic.reset_index(drop=True, inplace=True)
population_data_full.reset_index(drop=True, inplace=True)

  


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
population_data_basic #old

,year,name,value,datatypeid
0,1986,Budapest I. kerület,37275,1650
1,1986,Budapest II. kerület,102494,1650
2,1986,Budapest III. kerület,138796,1650
3,1986,Budapest IV. kerület,93247,1650
4,1986,Budapest IX. kerület,78399,1650
...,...,...,...,...
113397,2021,Zalatárnok,624,1650
113398,2021,Zalaújlak,102,1650
113399,2021,Zalavár,928,1650
113400,2021,Zalavég,335,1650


In [ ]:
population_data_full #old

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1986,Budapest I. kerület,37275,1650,9742,741,47.4965189917372,19.0347155108027
1,1986,Budapest II. kerület,102494,1650,9743,116,47.5444115110089,18.9751760210319
2,1986,Budapest III. kerület,138796,1650,9744,1579,47.5699972451916,19.0390024567837
3,1986,Budapest IV. kerület,93247,1650,1020,341,47.580097299462,19.0972915873203
4,1986,Budapest IX. kerület,78399,1650,15726,2701,47.468240300406,19.0934214369353
...,...,...,...,...,...,...,...,...
112933,2021,Zalatárnok,624,1650,10960,2711,46.7018614433717,16.758598572475
112934,2021,Zalaújlak,102,1650,17104,723,46.5607974894449,17.0798804702104
112935,2021,Zalavár,928,1650,10959,1150,46.6690051051186,17.1623405198731
112936,2021,Zalavég,335,1650,17327,1480,47.0077217561395,17.0278363053676


In [ ]:
#In both population_data_basic and population_data_full we subtract one from yea, because we want to the data match with the original export dataset coming later

population_data_basic.year = population_data_basic.year.astype(np.int64) - 1
population_data_basic = population_data_basic.applymap(str)

population_data_full.year = population_data_full.year.astype(np.int64) - 1
population_data_full = population_data_full.applymap(str)

In [ ]:
df1 = population_data_basic.merge(population_data_full[['year', 'name', 'value', 'datatypeid']], how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
df1 

,year,name,value,datatypeid,_merge
44,1985,Baranyahidvég,252,1650,left_only
55,1985,Birjan,364,1650,left_only
166,1985,Kovágószőlős,1255,1650,left_only
171,1985,Kővágótőttős,358,1650,left_only
286,1985,Szür,378,1650,left_only
...,...,...,...,...,...
67691,2006,Keleviz,385,1650,left_only
70843,2007,Hosszúviz,55,1650,left_only
70865,2007,Keleviz,383,1650,left_only
74017,2008,Hosszúviz,48,1650,left_only


In [ ]:
df1.name.unique() #Settlements that is contained in population_data_basic but not in population_data_full (Most of them are typos, so we need to fix that manually)

array(['Baranyahidvég', 'Birjan', 'Kovágószőlős', 'Kővágótőttős', 'Szür',
       'Tófü', 'Áprádhalom', 'Lászlófalva', 'Lökösháza', 'Szabadhidvég',
       'Felpérc', 'Lébénymiklós', 'Magyarfalva', 'Rábaújfalu',
       'Röjtökmuzsaly', 'Hadháztéglás', 'Leninváros', 'Csobaly',
       'Sajóhidvég', 'Tardosbánya', 'Berekdaróc', 'Jászalsószengyörgy',
       'Palotáshalom', 'Galgahéviz', 'Hévizgyörk', 'Lőrév', 'Verőcemaros',
       'Csombrád', 'Hosszúviz', 'Keleviz', 'Vizvár', 'Zselincszentpál',
       'Berecska', 'Csikóstöttös', 'Kaposszegcső', 'Möcsény', 'Mözs',
       'Gutatőttös', 'Megyehid', 'Nemesböd', 'Rábahidvég',
       'Bucsuszentlászló', 'Füzvölgy', 'Óhid', 'Pátró', 'Rádó', 'Lakhely'],
      dtype=object)

In [ ]:
df2 = population_data_full[['year', 'name', 'value', 'datatypeid']].merge(population_data_basic, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
df2

,year,name,value,datatypeid,_merge


In [ ]:
latLon_nan = population_data_full[population_data_full['wgs84_e_szelesseg'] == 'nan']
latLon_nan #All the instances that don't contain latitude and longitude

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
78,1985,Dobsza,1010,1650,10888,3182,nan,nan
194,1985,Mázaszászvár,4563,1650,17270,3187,nan,nan
678,1985,Bőnyrétalap,2770,1650,10397,3180,nan,nan
938,1985,Bodrogszegi,1510,1650,17230,3166,nan,nan
1368,1985,Almásneszmély,2967,1650,17046,3159,nan,nan
...,...,...,...,...,...,...,...,...
20194,1991,Kerepestarcsa,15962,1650,9976,3169,nan,nan
21033,1991,Sömjénmihályfa,1288,1650,10889,3174,nan,nan
21261,1991,Szemenyecsörnye,1260,1650,5491,3173,nan,nan
23281,1992,Kerepestarcsa,16037,1650,9976,3169,nan,nan


In [ ]:
latLon_nan.name.unique() 

array(['Dobsza', 'Mázaszászvár', 'Bőnyrétalap', 'Bodrogszegi',
       'Almásneszmély', 'Varsánygyüre', 'Kerepestarcsa', 'Boglárlelle',
       'Sömjénmihályfa', 'Szemenyecsörnye', 'Hajdúhadháztéglás'],
      dtype=object)

## To correct typos made in the population_data_basic dataset 

In [ ]:
population_data_basic = population_data_basic.replace('Baranyahidvég', 'Baranyahídvég')
population_data_basic = population_data_basic.replace('Birjan', 'Birján')
population_data_basic = population_data_basic.replace('Kovágószőlős', 'Kővágószőlős')
population_data_basic = population_data_basic.replace('Kővágótőttős', 'Kővágótöttös')
population_data_basic = population_data_basic.replace('Szür', 'Szűr')
population_data_basic = population_data_basic.replace('Tófü', 'Tófű')
population_data_basic = population_data_basic.replace('Áprádhalom', 'Árpádhalom')
population_data_basic = population_data_basic.replace('Lökösháza', 'Lőkösháza')
population_data_basic = population_data_basic.replace('Szabadhidvég', 'Szabadhídvég')
population_data_basic = population_data_basic.replace('Felpérc', 'Felpéc')
population_data_basic = population_data_basic.replace('Röjtökmuzsaly', 'Röjtökmuzsaj')
population_data_basic = population_data_basic.replace('Csobaly', 'Csobaj')
population_data_basic = population_data_basic.replace('Sajóhidvég', 'Sajóhídvég')
population_data_basic = population_data_basic.replace('Berekdaróc', 'Beregdaróc')
population_data_basic = population_data_basic.replace('Jászalsószengyörgy', 'Jászalsószentgyörgy')
population_data_basic = population_data_basic.replace('Galgahéviz', 'Galgahévíz')
population_data_basic = population_data_basic.replace('Hévizgyörk', 'Hévízgyörk')
population_data_basic = population_data_basic.replace('Lőrév', 'Lórév')
population_data_basic = population_data_basic.replace('Csombrád', 'Csombárd')
population_data_basic = population_data_basic.replace('Hosszúviz', 'Hosszúvíz')
population_data_basic = population_data_basic.replace('Keleviz', 'Kelevíz')
population_data_basic = population_data_basic.replace('Vizvár', 'Vízvár')
population_data_basic = population_data_basic.replace('Zselincszentpál', 'Zselicszentpál')
population_data_basic = population_data_basic.replace('Berecska', 'Belecska')
population_data_basic = population_data_basic.replace('Csikóstöttös', 'Csikóstőttős')
population_data_basic = population_data_basic.replace('Kaposszegcső', 'Kaposszekcső')
population_data_basic = population_data_basic.replace('Möcsény', 'Mőcsény')
population_data_basic = population_data_basic.replace('Megyehid', 'Megyehíd')
population_data_basic = population_data_basic.replace('Nemesböd', 'Nemesbőd')
population_data_basic = population_data_basic.replace('Rábahidvég', 'Rábahídvég')
population_data_basic = population_data_basic.replace('Bucsuszentlászló', 'Búcsúszentlászló')
population_data_basic = population_data_basic.replace('Füzvölgy', 'Fűzvölgy')
population_data_basic = population_data_basic.replace('Óhid', 'Óhíd')

In [ ]:
population_data_full = pd.merge(population_data_basic, geoname, on='name') #merged the fixed population_data_basic w/ geoname to get population_data_full

In [ ]:
population_data_full #Now it contains 113345 instances after fixing (before: 112938)

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1985,Budapest I. kerület,37275,1650,9742,741,47.4965189917372,19.0347155108027
1,1986,Budapest I. kerület,37086,1650,9742,741,47.4965189917372,19.0347155108027
2,1987,Budapest I. kerület,36641,1650,9742,741,47.4965189917372,19.0347155108027
3,1988,Budapest I. kerület,36302,1650,9742,741,47.4965189917372,19.0347155108027
4,1989,Budapest I. kerület,35956,1650,9742,741,47.4965189917372,19.0347155108027
...,...,...,...,...,...,...,...,...
113340,2016,Balatonakarattya,915,1650,10696,16544,47.0215257290545,18.1557677431017
113341,2017,Balatonakarattya,925,1650,10696,16544,47.0215257290545,18.1557677431017
113342,2018,Balatonakarattya,928,1650,10696,16544,47.0215257290545,18.1557677431017
113343,2019,Balatonakarattya,965,1650,10696,16544,47.0215257290545,18.1557677431017


In [ ]:
df1 = population_data_basic[['year', 'name', 'value', 'datatypeid']].merge(population_data_full[['year', 'name', 'value', 'datatypeid']], how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
len(df1) #it was 464 before now 57

57

In [ ]:
df1.name.unique() #Settlements that is contained in population_data_basic but not in population_data_full (much less, and those are settlements which nowadays have different names compared to the time before the change of the political system in Hungary)

array(['Lászlófalva', 'Lébénymiklós', 'Magyarfalva', 'Rábaújfalu',
       'Hadháztéglás', 'Leninváros', 'Tardosbánya', 'Palotáshalom',
       'Verőcemaros', 'Mözs', 'Gutatőttös', 'Pátró', 'Rádó', 'Lakhely'],
      dtype=object)

In [ ]:
export = pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/export.csv', dtype=str)
export = stringCol_mod(export) #Using Function2
datatypeid1650 = export[export['datatypeid'] == '1650'] #We are only interested in population data, so datatypeid=1650 is needed
datatypeid1650 = datatypeid1650.drop(datatypeid1650[datatypeid1650['geonameid'] == 'nan'].index) # Cleaning rows where geonameid == nan ind datatypeid1650 dataframe

In [ ]:
datatypeid1650.head() 

,year,value,geonameid,datatypeid
543,1993,16180,9976,1650
494177,1990,527,16358,1650
619479,1992,509,15896,1650
620770,1992,1791,16316,1650
621818,1992,704,9790,1650


In [ ]:
population_original = pd.merge(datatypeid1650, geoname, on='geonameid') #Getting the original population data from the data provided to us, that's why 'original'
print(len(population_original)) # #of instances in the population_original which contains population data from 1990 til 2016
print(len(population_data_full)) # #of instances in the population_data_full which contains population data from 1985 til 2020

84669
113345


In [ ]:
print(len(population_original[population_original.name.str.contains('Budapest')])) #population_original only contains Budapest wide population and 7-8 districts' populations --> 59 of them --> need to remove them because population_data_full conatins all districts' population.
population_original = population_original.drop(population_original[population_original.name.str.contains('Budapest')].index)
population_original.reset_index(drop=True, inplace=True)
print(len(population_original)) 

59
84610


In [ ]:
a = population_original[['name', 'year']]
b = population_data_full[['name', 'year']]
df_ab = a.merge(b, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
df_ab #getting those instances with year that are contained in population_original but not in population_data_full

,name,year,_merge
4,Kerepestarcsa,1994,left_only
22934,Nemesrádó,1990,left_only
58240,Tarjánpuszta,1992,left_only
61942,Tardos,1990,left_only
61956,Tardos,1991,left_only
63694,Sömjénmihályfa,1992,left_only
73829,Rábatöttös,1990,left_only
76179,Dobsza,1991,left_only
76183,Bőnyrétalap,1991,left_only
76189,Szemenyecsörnye,1992,left_only


In [ ]:
miss_idx = df_ab.index #getting the indecies of those instances that are contained in population_original but not in population_data_full
missing_items_from_population_data_full = population_original.iloc[miss_idx, :] #Use miss_idx as the indices of population_original to 
##create missing_items_from_population_data_full df that are going to be concatenated to the population_data_full df
len(missing_items_from_population_data_full)

14

In [ ]:
a = population_original[['name']]
b = population_data_full[['name']]
df_ba = b.merge(a, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
df_ba.name.unique() #getting those settlements that are contained in population_data_full but not in population_original (only expecting Budapest districts)

array(['Budapest I. kerület', 'Budapest II. kerület',
       'Budapest III. kerület', 'Budapest IV. kerület',
       'Budapest IX. kerület', 'Budapest V. kerület',
       'Budapest VI. kerület', 'Budapest VII. kerület',
       'Budapest VIII. kerület', 'Budapest X. kerület',
       'Budapest XI. kerület', 'Budapest XII. kerület',
       'Budapest XIII. kerület', 'Budapest XIV. kerület',
       'Budapest XIX. kerület', 'Budapest XV. kerület',
       'Budapest XVI. kerület', 'Budapest XVII. kerület',
       'Budapest XVIII. kerület', 'Budapest XX. kerület',
       'Budapest XXI. kerület', 'Budapest XXII. kerület',
       'Budapest XXIII. kerület'], dtype=object)

In [ ]:
#the created missing_items_from_population_data_full is going to be concatenated to the population_data_full df and population_data_basic
final_population = pd.concat([population_data_full, missing_items_from_population_data_full]) 
population_data_basic = pd.concat( [ population_data_basic, missing_items_from_population_data_full[['year', 'name', 'value', 'datatypeid']] ] )  

## Save it to CSV

In [ ]:
final_population.sort_values(by=['name', 'year'], inplace=True)
final_population.reset_index(drop=True, inplace=True)
final_population = final_population.replace('nan', np.nan)
final_population

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1985,Aba,4003,1650,9692,1512,47.0343556977497,18.524754962635
1,1986,Aba,4028,1650,9692,1512,47.0343556977497,18.524754962635
2,1987,Aba,4011,1650,9692,1512,47.0343556977497,18.524754962635
3,1988,Aba,4017,1650,9692,1512,47.0343556977497,18.524754962635
4,1989,Aba,4057,1650,9692,1512,47.0343556977497,18.524754962635
...,...,...,...,...,...,...,...,...
113354,2016,Ősi,2082,1650,9759,2160,47.1539446896795,18.190410278261
113355,2017,Ősi,2070,1650,9759,2160,47.1539446896795,18.190410278261
113356,2018,Ősi,2081,1650,9759,2160,47.1539446896795,18.190410278261
113357,2019,Ősi,2075,1650,9759,2160,47.1539446896795,18.190410278261


In [ ]:
population_data_basic.sort_values(by=['name', 'year'], inplace=True)
population_data_basic.reset_index(drop=True, inplace=True)
population_data_basic = population_data_basic.replace('nan', np.nan)
population_data_basic

,year,name,value,datatypeid
0,1985,Aba,4003,1650
1,1986,Aba,4028,1650
2,1987,Aba,4011,1650
3,1988,Aba,4017,1650
4,1989,Aba,4057,1650
...,...,...,...,...
113411,2016,Ősi,2082,1650
113412,2017,Ősi,2070,1650
113413,2018,Ősi,2081,1650
113414,2019,Ősi,2075,1650


In [ ]:
latLon_nan.sort_values(by=['name', 'year'], inplace=True)
latLon_nan.reset_index(drop=True, inplace=True)
latLon_nan = latLon_nan.replace('nan', np.nan)
latLon_nan

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1985,Almásneszmély,2967,1650,17046,3159,NaN,NaN
1,1986,Almásneszmély,2968,1650,17046,3159,NaN,NaN
2,1987,Almásneszmély,2964,1650,17046,3159,NaN,NaN
3,1988,Almásneszmély,2933,1650,17046,3159,NaN,NaN
4,1989,Almásneszmély,2917,1650,17046,3159,NaN,NaN
...,...,...,...,...,...,...,...,...
65,1986,Varsánygyüre,2931,1650,15983,3179,NaN,NaN
66,1987,Varsánygyüre,2920,1650,15983,3179,NaN,NaN
67,1988,Varsánygyüre,2913,1650,15983,3179,NaN,NaN
68,1989,Varsánygyüre,2892,1650,15983,3179,NaN,NaN


In [ ]:
population_data_basic.to_csv(output_path + 'populationData_HU_1985-2020_basic.csv', index = False)
final_population.to_csv(output_path + 'populationData_HU_1985-2020_full.csv', index = False)
latLon_nan.to_csv(output_path + 'latLon_empty.csv', index = False)
df1.to_csv(output_path + 'basic_full_diff.csv', index = False) #dataset with instances that are left out of the final_population but contained in population_data_basic